In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

#### Review Scraping

In [ ]:
# get all the pages with restaurants in New York
ny_response = requests.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York%2C+NY&ns=1')
ny_results = BeautifulSoup(ny_response.content,'html')
ny_pagenum = ny_results.find('div', {'role':"navigation"}).find('span', {'class':
    "lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_"}).text[10:]
ny_url = []
for i in range(int(ny_pagenum)):
    ny_url.append('https://www.yelp.com/search?find_desc=Restaurants&find_loc=New%20York%2C%20NY&ns=1%3Fstart%3D960' + f'&start={30*i}')

# get each restaurant's link
res_link = []
for i in ny_url[:5]:
    payload = {'api_key': '74eed888a9a7e4d48e3e2646387e3e97', 'url': i}
    page_response = requests.get('http://api.scraperapi.com', params = payload)
    try:
        page_response.status_code == 200
        print("Scraping success")
        page_results = BeautifulSoup(page_response.content,'html')
            
        meta = page_results.find('meta', {'property': 'og:description'}) # get each page's restaurants' name
        res_list = meta.attrs['content'].split('NY - ')[1].split(',')

        res_all = []
        for j in res_list:
            res_all.append(j.strip().replace('\'','’'))
        for l in res_all:
            res_link.append((l, url + page_results.find('a', {'name':"%s"%(l)}).get('href')))
    except:
        print("Scraping error")

# define get recommended review function
def get_review(res_link: list):
    reviews = []
    for item in res_link:
        review = []
        name = item[0]
        link = item[1]
        payload = {'api_key': '74eed888a9a7e4d48e3e2646387e3e97', 'url': link}
        res_response = requests.get('http://api.scraperapi.com', params = payload)
        try:
            res_response.status_code == 200
            print("Outside scraping success")
            res_results = BeautifulSoup(res_response.content,'html')

            review.extend(json.loads(res_results.find('script', {'type':"application/ld+json"}).text)['review']) # get 1st page review

            res_pagenum = res_results.find('div',{'role':"navigation"}).find('span', {'class': 
                    "lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_"}).get_text()[10:]
            if res_pagenum != 1:   
                review_link = []
                for i in range(1,int(res_pagenum)):
                    review_link.append('https://www.yelp.com/biz/am%C3%A9lie-new-york?osq=Restaurants' + f'&start={20*i}') # get the remaining review pages' links
                for j in review_link:
                    payload = {'api_key': '74eed888a9a7e4d48e3e2646387e3e97', 'url': j}
                    next_res_response = requests.get('http://api.scraperapi.com', params = payload)
                    try:
                        next_res_response.status_code == 200
                        print('Inside scraping success')
                        next_res_results = BeautifulSoup(next_res_response.content,'html')
                        review.extend(json.loads(next_res_results.find('script', {'type':"application/ld+json"}).text)['review'])
                        max_num = str((int(res_pagenum)-1)*20)
                        if max_num in j:
                            break
                        if len(review) >= 200: # scrap 200 reviews of each restaurant
                            break
                    except:
                        print("Inside scraping error")
                        raise
            else:
                pass
            for l in review:
                    l['reviewRating'] = l['reviewRating']['ratingValue']
                    l['businessName'] = name
            reviews.extend(review)
        except:
            print("Outside scraping error")
            raise
    return reviews

# get all the recommended reviews
get_review(res_link)

# convert them into dataframe
df = pd.DataFrame(recommended_review)
df['label'] = 1 # set these reviews as true reviews
df.columns = ['rating', 'date', 'review', 'author', 'business_name', 'label']

# not recommended review scraping is following the same algorithms
# the only difference is the links and the attributes of data in web pages